In [ ]:
%mkdir -p /csc413/project/
%cd /csc413/project
%pip install pytorch-fid

import os
import sys
import cv2
import shutil
import torch
import pickle
import zipfile
import torchvision
import pytorch_fid
import numpy as np
from PIL import Image
from typing import Any
import tensorflow as tf
import tensorflow_hub as hub
from skimage.metrics import structural_similarity as compare_ssim
from six.moves.urllib.request import urlretrieve
import matplotlib.pyplot as plt

/csc413/project
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def download(zip_to_download_name, filename):
  origin = "https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/"
  origin += zip_to_download_name

  datadir = os.path.join("data")
  if not os.path.exists(datadir):
      os.makedirs(datadir)

  zip_path = os.path.join(datadir, filename)
  zip_path = zip_path + ".zip"

  urlretrieve(origin, zip_path)

  with zipfile.ZipFile(zip_path,"r") as zip_ref:
      zip_ref.extractall(datadir)

In [ ]:
download("vangogh2photo.zip", "vangogh")
download("ukiyoe2photo.zip", "ukiyoe")
download("monet2photo.zip", "monet")

In [ ]:
if os.path.exists("/CSC413_test_data.zip"):
  %mv /CSC413_test_data.zip /csc413/project/CSC413_test_data.zip
elif os.path.exists("/content/CSC413_test_data.zip"):
  %mv /content/CSC413_test_data.zip /csc413/project/CSC413_test_data.zip

In [ ]:
zip_path = "/csc413/project/CSC413_test_data.zip"
datadir = "/csc413/project/"
with zipfile.ZipFile(zip_path,"r") as zip_ref:
      zip_ref.extractall(datadir)

FileNotFoundError: ignored

# Setup

run the following code to load global variables and import required modules

In [ ]:
# Define the size of the generated image
img_size = 256

# Define the weights to use for style mixing
alpha = 0.5
beta = 1 - alpha

# Model URLs
NST_Pretrained = "https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/1"

# Data Loading

The following code load data for Model usage

In [ ]:
# Define a function to load and preprocess images
def load_image(img_path: str) -> Any:
    """
    Load and preprocess the image.

    :param img_path: the path of image to load
    :return: the processed image
    """
    img = plt.imread(img_path)
    img = np.array(img).astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# NST

In [ ]:
class NST:
    """
    The styleGAN pretrained-model.

    Instance Variables:
        - url: the url of the model
        - model: the model of the pretrained model
        - mixed_features: the mixed feature after mixing style 1 and 2
    """
    _url: str
    _model: Any
    _mixed_features: Any

    def __init__(self, url: str) -> None:
        """
        Initialization

        :param url: the url of the pretrained_model
        """
        self._url = url

    def _load_model(self) -> None:
        """
        load the model
        """
        self._model = hub.load(self._url)


    def _mix(self, style1, style2) -> None:
        """
        Mix the style of style1 and style2 corresponding to hyperparameter alpha and beta
        and store the style into self.mixed_features

        :param style1: source of style1
        :param style2: source of style2
        """
        self._mixed_features = self._model(tf.constant(style1), tf.constant(style2))[0]
        self._mixed_features = self._mixed_features.numpy()

    def _generate_and_save(self, output_path) -> None:
        """
        Use self.model and self.mixed_features to generate the mixed pictures and store them
        """
        img = self._mixed_features[0]
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = (img * 255).astype(np.uint8)
        cv2.imwrite((output_path), img)

    def generate(self, input, style, output_path) -> None:
        """
        Now perform image generation using the model!

        :param input: source of input
        :param style: source of style
        """
        self._load_model()
        self._mix(input, style)
        self._generate_and_save(output_path)

In [ ]:
def rename_test_source(source):
  source_dir = f"test data/test_{source}"

  source_lst = os.listdir(source_dir)
  print(source_lst)

  for i in range(len(source_lst)):
        source_path = os.path.join(source_dir, source_lst[i])
        print(f"{source} {i + 1} {source_path}")
        os.rename(source_path, os.path.join(source_dir, f"source_{source}_{i + 1}.jpg"))

In [ ]:
def run_model(model, input, style, save_source=False): 
  # create folder to save output
  output_dir = f"out_image/{input}_to_{style}"
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  # get path for input and style
  if input == "photo":
    test_input_dir = f"test data/test_{input}"
  else:
    test_input_dir = f"out_image/photo_to_{input}"
    if not os.path.exists(test_input_dir):
      print(f"cannot find result from first style transfer (photo -> {input})")
      return
  test_style_dir = f"test data/test_{style}"
  
  test_input_lst = sorted(os.listdir(test_input_dir))
  test_style_lst = sorted(os.listdir(test_style_dir))
  
  # perform style transfer for each pair of photo and style
  for i in range(len(test_input_lst)):
    input_path = os.path.join(test_input_dir, test_input_lst[i])
    print(f"{i + 1} input_path: {input_path}")
    style_path = os.path.join(test_style_dir, test_style_lst[i])
    print(f"{i + 1} style_path: {style_path}")

    input_img = load_image(input_path)
    style_img = load_image(style_path)

    output_path = os.path.join(output_dir, f"out_{input}_to_{style}_{i + 1}.jpg")

    model.generate(input_img, style_img, output_path)

    if (save_source): 
      shutil.copy(input_path, output_dir)
      shutil.copy(style_path, output_dir)
      os.rename(os.path.join(output_dir, test_input_lst[i]), os.path.join(output_dir, f"source_{input}_{i + 1}.jpg"))
      os.rename(os.path.join(output_dir, test_style_lst[i]), os.path.join(output_dir, f"source_{style}_{i + 1}.jpg"))

    print(f"done {input} to {style} {i + 1}")

In [ ]:
model = NST(NST_Pretrained)

rename_test_source("photo")
rename_test_source("monet")
rename_test_source("vangogh")

In [ ]:
run_model(model, "photo", "monet")
run_model(model, "photo", "vangogh")

In [ ]:
run_model(model, "monet", "vangogh")
run_model(model, "vangogh", "monet")

In [ ]:
def show_plot(figure, image, title, data_idx, plot_idx):
  figure.add_subplot(nrows, ncols, plot_idx)
  plt.title(f"{title}_{data_idx}")
  plt.axis('off')
  plt.imshow(image)

In [ ]:
i = 3
nrows = 3
ncols = 3

fig = plt.figure(figsize=(10, 10))

M = Image.open(f"test data/test_monet/source_monet_{i}.jpg")
P = Image.open(f"test data/test_photo/source_photo_{i}.jpg")
V = Image.open(f"test data/test_vangogh/source_vangogh_{i}.jpg")
PtM = Image.open(f"out_image/photo_to_monet/out_photo_to_monet_{i}.jpg")
PtV = Image.open(f"out_image/photo_to_vangogh/out_photo_to_vangogh_{i}.jpg")
MtV = Image.open(f"out_image/monet_to_vangogh/out_monet_to_vangogh_{i}.jpg")
VtM = Image.open(f"out_image/vangogh_to_monet/out_vangogh_to_monet_{i}.jpg")


show_plot(fig, M, "source_monet", i, 1)
show_plot(fig, P, "source_photo", i, 2)
show_plot(fig, V, "source_vangogh", i, 3)

show_plot(fig, PtM, "photo_monet", i, 4)
show_plot(fig, PtV, "photo_vangogh", i, 6)

show_plot(fig, MtV, "monet_vangogh", i, 7)
show_plot(fig, VtM, "vangogh_monet", i, 9)